**Problem 2**

Speech Denoising

In [0]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import math
import numpy as np
import random
import librosa
import sklearn

In [0]:
epsilon = 1e-3

test_1 = 'test_x_01.wav'
output_1 = 'output_1.wav'

test_2 = 'test_x_02.wav'
output_2 = 'output_2.wav'

In [0]:
x = tf.placeholder("float", [None, 513])
y = tf.placeholder("float", [None, 513])

n_hidden_1 = 1000
n_hidden_2 = 1000
op_layer = 513
num_input = 513 
num_output = 513 

save_path = "speech_denoising.ckpt/"

In [4]:
st_dev = math.sqrt(2/1000)

# Store layers weight & bias
weights = {
    'ip': tf.Variable(tf.truncated_normal([num_input, n_hidden_1], stddev = st_dev)),
    'h1': tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2], stddev = st_dev)),
    'h2': tf.Variable(tf.truncated_normal([n_hidden_2, op_layer], stddev = st_dev)),
    'op': tf.Variable(tf.truncated_normal([op_layer, num_output], stddev = st_dev))
}

biases = {
    'ip': tf.Variable(tf.zeros([n_hidden_1])),
    'b1': tf.Variable(tf.zeros([n_hidden_2])),
    'b2': tf.Variable(tf.zeros([op_layer])),
    'op': tf.Variable(tf.zeros([num_output]))
}


Instructions for updating:
Colocations handled automatically by placer.


**Neural Network structure:**

2 hidden layers with 1000 neurons in each. Activation function for all is Relu and layer normalization done for all layers.

In [0]:
Wb_ip = tf.add(tf.matmul(x, weights['ip']), biases['ip'])
#layer_ip = tf.nn.relu(Wb_ip)
mean_ip, var_ip = tf.nn.moments(Wb_ip, [0])
scale_ip = tf.Variable(tf.ones([n_hidden_1]))
b_ip = tf.Variable(tf.zeros([n_hidden_1]))
layer_ip = tf.nn.relu(tf.nn.batch_normalization(Wb_ip, mean_ip, var_ip, b_ip, scale_ip, epsilon))
    
Wb_1 = tf.add(tf.matmul(layer_ip, weights['h1']), biases['b1'])
#layer_1 = tf.nn.relu(Wb_1)
mean_1, var_1 = tf.nn.moments(Wb_1, [0])
scale_1 = tf.Variable(tf.ones([n_hidden_2]))
b_1 = tf.Variable(tf.zeros([n_hidden_2]))
layer_1 = tf.nn.relu(tf.nn.batch_normalization(Wb_1, mean_1, var_1, b_1, scale_1, epsilon))

Wb_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
mean_2, var_2 = tf.nn.moments(Wb_2, [0])
scale_2 = tf.Variable(tf.ones([op_layer]))
b_2 = tf.Variable(tf.zeros([op_layer]))
layer_2 = tf.nn.relu(tf.nn.batch_normalization(Wb_2, mean_2, var_2, b_2, scale_2, epsilon))

output_layer = tf.add(tf.matmul(layer_2, weights['op']), biases['op'])

Decaying learning rate gave the best output

In [6]:
loss=tf.losses.mean_squared_error(labels=y,predictions=output_layer)


step = tf.Variable(0, trainable=False)
rate = tf.train.exponential_decay(0.01, step, 1, 0.9999)

train = tf.train.AdamOptimizer(learning_rate=rate).minimize(loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


In [0]:
def train_model():
    
    
    s, sr = librosa.load('train_clean_male.wav', sr=None)
    S = librosa.stft(s, n_fft=1024, hop_length=512)
    sn, sr = librosa.load('train_dirty_male.wav', sr=None)
    X = librosa.stft(sn, n_fft=1024, hop_length=512)
    
    S = np.abs(S)
    S = S.T
    X = np.abs(X)
    X = X.T
    
    saver=tf.train.Saver()
      
    for i in range(10001):
        X_shuffled,S_shuffled = sklearn.utils.shuffle(X,S)
        X_batch = X_shuffled[0:30,:]
        S_batch = S_shuffled[0:30,:]
        
        _,error = sess.run([train,loss],feed_dict={x:X_batch, y:S_batch})        
        if i % 100 == 0:
            print("Epoch: ",i/100)
            saver.save(sess, save_path)

    print("Error = ", error)

In [8]:
train_model()

Epoch:  0.0
Epoch:  1.0
Epoch:  2.0
Epoch:  3.0
Epoch:  4.0
Epoch:  5.0
Epoch:  6.0
Epoch:  7.0
Epoch:  8.0
Epoch:  9.0
Epoch:  10.0
Epoch:  11.0
Epoch:  12.0
Epoch:  13.0
Epoch:  14.0
Epoch:  15.0
Epoch:  16.0
Epoch:  17.0
Epoch:  18.0
Epoch:  19.0
Epoch:  20.0
Epoch:  21.0
Epoch:  22.0
Epoch:  23.0
Epoch:  24.0
Epoch:  25.0
Epoch:  26.0
Epoch:  27.0
Epoch:  28.0
Epoch:  29.0
Epoch:  30.0
Epoch:  31.0
Epoch:  32.0
Epoch:  33.0
Epoch:  34.0
Epoch:  35.0
Epoch:  36.0
Epoch:  37.0
Epoch:  38.0
Epoch:  39.0
Epoch:  40.0
Epoch:  41.0
Epoch:  42.0
Epoch:  43.0
Epoch:  44.0
Epoch:  45.0
Epoch:  46.0
Epoch:  47.0
Epoch:  48.0
Epoch:  49.0
Epoch:  50.0
Epoch:  51.0
Epoch:  52.0
Epoch:  53.0
Epoch:  54.0
Epoch:  55.0
Epoch:  56.0
Epoch:  57.0
Epoch:  58.0
Epoch:  59.0
Epoch:  60.0
Epoch:  61.0
Epoch:  62.0
Epoch:  63.0
Epoch:  64.0
Epoch:  65.0
Epoch:  66.0
Epoch:  67.0
Epoch:  68.0
Epoch:  69.0
Epoch:  70.0
Epoch:  71.0
Epoch:  72.0
Epoch:  73.0
Epoch:  74.0
Epoch:  75.0
Epoch:  76.0
Epoch:  7

In [9]:
def test_model(test_file, output_file):
    print("Testing :", test_file)
    sn, sr = librosa.load(test_file, sr=None)
    X = librosa.stft(sn, n_fft=1024, hop_length=512)
    T = X
    batch_x = np.transpose(np.abs(X))
    
    output = sess.run(output_layer, feed_dict={x: batch_x})
    output_t = np.transpose(output)
    
    final_answer = np.multiply(np.divide(T, np.abs(T)), output_t)
    final_answer = librosa.istft(final_answer, win_length=1024, hop_length=512)
    librosa.output.write_wav(output_file, final_answer, sr)
    
test_model(test_1,output_1)
test_model(test_2,output_2)
sess.close()

Testing : test_x_01.wav
Testing : test_x_02.wav


Cleaned files are generated and can be played.

In [10]:
import IPython.display as ipd
ipd.Audio('./output_1.wav')

In [11]:
import IPython.display as ipd
ipd.Audio('./output_2.wav')